In [103]:
import pandas as pd

In [104]:
df_produkcja = pd.read_csv('electricity_production_entsoe_all.csv')
df_fg = pd.read_csv('fg_data_for_location.csv')
df_maxProdukcja = pd.read_csv('installed_capacity_entsoe_all.csv')

In [105]:
df_produkcja = df_produkcja[['date', 'wind_onshore']]
df_produkcja['date'] = pd.to_datetime(df_produkcja['date'], format='%d.%m.%Y %H:%M', errors='coerce')
df_produkcja.set_index('date', inplace=True)
# Grupowanie po godzinie i obliczanie średniej dla każdej godziny
df_produkcja = df_produkcja.resample('D').mean()
df_produkcja.dropna(inplace=True)
df_produkcja.head()

,wind_onshore
date,
2015-01-02,3233.791667
2015-01-03,3374.375000
2015-01-04,3212.333333
2015-01-05,1804.541667
2015-01-06,770.791667


In [106]:
df_fg.head()

,Date,Wind Speed
0,2011-01-01 00:00:00,4.35
1,2011-01-02 00:00:00,4.88
2,2011-01-03 00:00:00,1.76
3,2011-01-04 00:00:00,1.44
4,2011-01-05 00:00:00,1.85


In [107]:
df_maxProdukcja['windMax'] = df_maxProdukcja['wind']
df_maxProdukcja = df_maxProdukcja[['date', 'windMax']]
df_maxProdukcja.head()

,date,windMax
0,2015,3758.0
1,2016,5494.0
2,2017,6026.0
3,2018,5791.0
4,2019,5808.0


In [108]:
df_produkcja.dtypes

wind_onshore    float64
dtype: object

In [109]:
df_fg.dtypes

Date           object
Wind Speed    float64
dtype: object

In [110]:

df_fg['datetime'] = pd.to_datetime(df_fg['Date'], errors='coerce')
df_fg['year'] = df_fg['datetime'].dt.year
df_fg.set_index('datetime', inplace=True)

In [111]:
# Połącz dwa zestawy danych na podstawie indeksu
df_combined = df_produkcja.join(df_fg, how='inner') 
df_combined.reset_index(inplace=True)


In [112]:
df_combined.head()

,index,wind_onshore,Date,Wind Speed,year
0,2015-01-02,3233.791667,2015-01-02 00:00:00,5.18,2015
1,2015-01-03,3374.375000,2015-01-03 00:00:00,5.71,2015
2,2015-01-04,3212.333333,2015-01-04 00:00:00,4.06,2015
3,2015-01-05,1804.541667,2015-01-05 00:00:00,2.24,2015
4,2015-01-06,770.791667,2015-01-06 00:00:00,1.88,2015


In [113]:
df_combined = pd.merge(df_combined, df_maxProdukcja[['date', 'windMax']],  left_on='year', right_on='date', how='left')

df_combined['efficiency'] = df_combined['wind_onshore'] / df_combined['windMax']


In [114]:
df_combined.head()

,index,wind_onshore,Date,Wind Speed,year,date,windMax,efficiency
0,2015-01-02,3233.791667,2015-01-02 00:00:00,5.18,2015,2015,3758.0,0.860509
1,2015-01-03,3374.375000,2015-01-03 00:00:00,5.71,2015,2015,3758.0,0.897918
2,2015-01-04,3212.333333,2015-01-04 00:00:00,4.06,2015,2015,3758.0,0.854799
3,2015-01-05,1804.541667,2015-01-05 00:00:00,2.24,2015,2015,3758.0,0.480187
4,2015-01-06,770.791667,2015-01-06 00:00:00,1.88,2015,2015,3758.0,0.205107


In [115]:
df_combined.reset_index(inplace=True) 

# df_combined.head()

In [116]:
print(df_combined.columns)

Index(['level_0', 'index', 'wind_onshore', 'Date', 'Wind Speed', 'year',
       'date', 'windMax', 'efficiency'],
      dtype='object')


In [117]:
df_combined.head()

,level_0,index,wind_onshore,Date,Wind Speed,year,date,windMax,efficiency
0,0,2015-01-02,3233.791667,2015-01-02 00:00:00,5.18,2015,2015,3758.0,0.860509
1,1,2015-01-03,3374.375000,2015-01-03 00:00:00,5.71,2015,2015,3758.0,0.897918
2,2,2015-01-04,3212.333333,2015-01-04 00:00:00,4.06,2015,2015,3758.0,0.854799
3,3,2015-01-05,1804.541667,2015-01-05 00:00:00,2.24,2015,2015,3758.0,0.480187
4,4,2015-01-06,770.791667,2015-01-06 00:00:00,1.88,2015,2015,3758.0,0.205107


In [118]:
df_combined = df_combined[['index', 'Wind Speed', 'efficiency']]
df_combined.max()

index         2024-06-30 00:00:00
Wind Speed                   8.59
efficiency               1.069119
dtype: object

In [119]:

df_combined.head()

,index,Wind Speed,efficiency
0,2015-01-02,5.18,0.860509
1,2015-01-03,5.71,0.897918
2,2015-01-04,4.06,0.854799
3,2015-01-05,2.24,0.480187
4,2015-01-06,1.88,0.205107


In [120]:
# Zapis do pliku CSV
df_combined.to_csv('datasetWind.csv', index=False)  # index=False, aby nie zapisywać indeksu

print("Plik CSV został zapisany jako 'datasetWind.csv'")

Plik CSV został zapisany jako 'datasetWind.csv'
